# Pipeline de Metatranscriptômica (Viroma)

## Objetivo
Processar os arquivos FASTQ brutos da amostra do paciente para identificar organismos presentes (principalmente vírus), confirmar a presença de SARS-CoV-2 e avaliar coinfecções.

## Passos do Pipeline
1. Controle de Qualidade (FastQC + MultiQC)
2. Pré-processamento/Trimming (fastp)
3. Remoção de Reads do Hospedeiro (Bowtie2)
4. Classificação Taxonômica (Kraken2 + Bracken)
5. Visualização (Krona ou Python)

In [11]:
# Configuração inicial
import os
import sys
from pathlib import Path

# Obter diretório raiz do projeto (assumindo que notebook está em notebooks/)
# Se notebook está em notebooks/, então parent.parent é a raiz
notebook_dir = Path().resolve()  # Diretório atual de trabalho
if 'notebooks' in str(notebook_dir):
    # Se estamos em notebooks/, subir 1 nível
    project_root = notebook_dir.parent
else:
    # Se executando de outro lugar, tentar encontrar a raiz
    # Ou usar caminho absoluto se necessário
    project_root = Path('/Users/larissa/Desktop/TCC_metatrascriptomica')

# Adicionar scripts ao path
sys.path.insert(0, str(project_root))

# Verificar se scripts existe
scripts_dir = project_root / "scripts"
if not scripts_dir.exists():
    raise FileNotFoundError(
        f"Diretório 'scripts' não encontrado em {project_root}.\n"
        f"Diretório atual de trabalho: {Path().resolve()}\n"
        f"Tente executar: os.chdir('{project_root / 'notebooks'}')"
    )

# Configurar caminhos relativos
DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
RESULTS_DIR = project_root / "results"
QC_REPORTS_DIR = RESULTS_DIR / "qc_reports"

# Criar diretórios se não existirem
for directory in [RAW_DIR, PROCESSED_DIR, RESULTS_DIR, QC_REPORTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Mudar para diretório do notebook (opcional, mas útil)
os.chdir(project_root / "notebooks")

print(f"Project root: {project_root}")
print(f"Data directory: {DATA_DIR}")
print(f"Scripts directory: {scripts_dir} {'✅' if scripts_dir.exists() else '❌'}")
print(f"Diretório de trabalho: {os.getcwd()}")

Project root: /Users/larissa/Desktop/TCC_metatrascriptomica
Data directory: /Users/larissa/Desktop/TCC_metatrascriptomica/data
Scripts directory: /Users/larissa/Desktop/TCC_metatrascriptomica/scripts ✅
Diretório de trabalho: /Users/larissa/Desktop/TCC_metatrascriptomica/notebooks


## 1. Controle de Qualidade

In [12]:
# Importar funções dos scripts
from scripts.qc_processing import run_fastqc, generate_multiqc_report

# Definir arquivos de entrada
r1_file = str(RAW_DIR / "patient_joao_VIROMA_S21_R1_001.fastq.gz")
r2_file = str(RAW_DIR / "patient_joao_VIROMA_S21_R2_001.fastq.gz")

# Verificar se arquivos existem
import os
if not os.path.exists(r1_file):
    print(f"⚠️ ATENÇÃO: Arquivo R1 não encontrado: {r1_file}")
if not os.path.exists(r2_file):
    print(f"⚠️ ATENÇÃO: Arquivo R2 não encontrado: {r2_file}")

# Executar FastQC
if os.path.exists(r1_file) and os.path.exists(r2_file):
    print("Executando FastQC...")
    fastqc_results = run_fastqc([r1_file, r2_file], output_dir=str(QC_REPORTS_DIR))
    print("FastQC concluído!")

    # Gerar relatório MultiQC
    print("Gerando relatório MultiQC...")
    multiqc_report = generate_multiqc_report(qc_dir=str(QC_REPORTS_DIR))
else:
    print("❌ Não é possível executar FastQC: arquivos de entrada não encontrados.")

Executando FastQC...
Executando FastQC em: /Users/larissa/Desktop/TCC_metatrascriptomica/data/raw/patient_joao_VIROMA_S21_R1_001.fastq.gz
Executando FastQC em: /Users/larissa/Desktop/TCC_metatrascriptomica/data/raw/patient_joao_VIROMA_S21_R2_001.fastq.gz
FastQC concluído!
Gerando relatório MultiQC...
Gerando relatório MultiQC...
Relatório MultiQC gerado: results/qc_reports/multiqc_report.html


## 2. Pré-processamento (Trimming)

In [13]:
from scripts.trimming import trim_reads_fastp

# Definir arquivos de saída
output_r1 = str(PROCESSED_DIR / "patient_joao_R1_trimmed.fastq.gz")
output_r2 = str(PROCESSED_DIR / "patient_joao_R2_trimmed.fastq.gz")

# Executar trimming
if os.path.exists(r1_file) and os.path.exists(r2_file):
    print("Executando trimming com fastp...")
    trim_results = trim_reads_fastp(
        input_r1=r1_file,
        input_r2=r2_file,
        output_r1=output_r1,
        output_r2=output_r2,
        output_dir=str(PROCESSED_DIR)
    )
    print("Trimming concluído!")
else:
    print("❌ Não é possível executar trimming: arquivos de entrada não encontrados.")

Executando trimming com fastp...
Executando fastp em patient_joao_VIROMA_S21_R1_001.fastq.gz e patient_joao_VIROMA_S21_R2_001.fastq.gz...
Trimming concluído. Outputs salvos em:
  R1: /Users/larissa/Desktop/TCC_metatrascriptomica/data/processed/patient_joao_R1_trimmed.fastq.gz
  R2: /Users/larissa/Desktop/TCC_metatrascriptomica/data/processed/patient_joao_R2_trimmed.fastq.gz
Trimming concluído!


## 3. Remoção de Reads do Hospedeiro

In [14]:
from scripts.host_removal import remove_human_reads_bowtie2

# Caminho para índice do genoma humano
# NOTA: Se ainda não criou o índice, pode pular esta etapa ou criar antes
human_ref_index = str(DATA_DIR / "references" / "hg38_index")

# Verificar se índice existe
index_exists = os.path.exists(f"{human_ref_index}.1.bt2") or os.path.exists(f"{human_ref_index}.bt2")

# Arquivos de saída (reads não-humanas)
nonhuman_r1 = str(PROCESSED_DIR / "patient_joao_nonhuman_R1.fastq.gz")
nonhuman_r2 = str(PROCESSED_DIR / "patient_joao_nonhuman_R2.fastq.gz")

# Executar remoção de reads humanas
if index_exists and os.path.exists(output_r1) and os.path.exists(output_r2):
    print("Executando remoção de reads do hospedeiro com Bowtie2...")
    host_removal_results = remove_human_reads_bowtie2(
        input_r1=output_r1,
        input_r2=output_r2,
        reference_index=human_ref_index,
        output_r1=nonhuman_r1,
        output_r2=nonhuman_r2,
        threads=8
    )
    print("Remoção de reads do hospedeiro concluída!")
elif not index_exists:
    print(f"⚠️ Índice do genoma humano não encontrado em: {human_ref_index}")
    print("   Pulando etapa de remoção de hospedeiro. Usando reads trimmed diretamente.")
    # Usar reads trimmed como se fossem non-human
    nonhuman_r1 = output_r1
    nonhuman_r2 = output_r2
else:
    print("❌ Não é possível executar remoção de hospedeiro: arquivos trimmed não encontrados.")

Executando remoção de reads do hospedeiro com Bowtie2...
Removendo reads do hospedeiro com Bowtie2...
  Input: patient_joao_R1_trimmed.fastq.gz, patient_joao_R2_trimmed.fastq.gz
  Reference: /Users/larissa/Desktop/TCC_metatrascriptomica/data/references/hg38_index
Remoção de reads do hospedeiro concluída.
  Output R1: /Users/larissa/Desktop/TCC_metatrascriptomica/data/processed/patient_joao_nonhuman_R1.fastq.gz
  Output R2: /Users/larissa/Desktop/TCC_metatrascriptomica/data/processed/patient_joao_nonhuman_R2.fastq.gz
Remoção de reads do hospedeiro concluída!


## 4. Classificação Taxonômica (Kraken2 + Bracken)

In [15]:
from scripts.taxonomic_classification import run_kraken2, run_bracken, parse_kraken2_report, parse_bracken_report
import pandas as pd

# Caminho para banco de dados Kraken2
kraken_db = "/Users/larissa/kraken2_db/viral_db"
kraken_db_expanded = os.path.expanduser(kraken_db)

# Verificar se banco existe
if not os.path.exists(kraken_db_expanded):
    print(f"⚠️ Banco Kraken2 não encontrado em: {kraken_db_expanded}")
    print("   Verifique o caminho ou baixe o banco seguindo o guia.")

# Criar diretório de relatórios
kraken_reports_dir = RESULTS_DIR / "kraken2_reports"
kraken_reports_dir.mkdir(parents=True, exist_ok=True)

# Arquivos de saída
kraken_output = str(kraken_reports_dir / "patient_joao_kraken2_output.txt")
kraken_report = str(kraken_reports_dir / "patient_joao_kraken2_report.txt")
bracken_output = str(kraken_reports_dir / "patient_joao_bracken_output.txt")

# Executar Kraken2
if os.path.exists(kraken_db_expanded) and os.path.exists(nonhuman_r1) and os.path.exists(nonhuman_r2):
    print("Executando Kraken2 (isso pode demorar alguns minutos)...")
    kraken_results = run_kraken2(
        input_r1=nonhuman_r1,
        input_r2=nonhuman_r2,
        db_path=kraken_db_expanded,
        output_file=kraken_output,
        report_file=kraken_report,
        threads=8
    )
    print("Kraken2 concluído!")

    # Executar Bracken (opcional - se não estiver instalado, usar apenas Kraken2)
    if os.path.exists(kraken_report):
        # Verificar se Bracken está disponível
        import shutil
        bracken_available = shutil.which("bracken") is not None
        
        if bracken_available:
            try:
                print("Executando Bracken...")
                bracken_results = run_bracken(
                    kraken_report=kraken_report,
                    db_path=kraken_db_expanded,
                    output_file=bracken_output
                )
                print("Bracken concluído!")
            except Exception as e:
                print(f"⚠️ Erro ao executar Bracken: {e}")
                print("   Continuando com resultados do Kraken2 apenas...")
                bracken_output = None
        else:
            print("⚠️ Bracken não está instalado ou não está no PATH.")
            print("   Usando resultados do Kraken2 apenas (isso é suficiente para o pipeline).")
            print("   Para instalar Bracken: conda install -c bioconda bracken")
            bracken_output = None

    # Parsear e visualizar resultados
    if os.path.exists(kraken_report):
        print("\n📊 Parseando resultados...")
        kraken_df = parse_kraken2_report(kraken_report)
        print(f"Total de táxons identificados: {len(kraken_df)}")
        
        # Filtrar espécies (rank == 'S') e mostrar top 10
        species_df = kraken_df[kraken_df['rank'] == 'S'].nlargest(10, 'reads')
        print("\nTop 10 Espécies Detectadas:")
        print(species_df[['name', 'reads', 'percent']].to_string(index=False))
        
        if bracken_output and os.path.exists(bracken_output):
            bracken_df = parse_bracken_report(bracken_output)
            print(f"\nBracken: Total de espécies estimadas: {len(bracken_df)}")
        else:
            print("\n⚠️ Resultados do Bracken não disponíveis (usando apenas Kraken2)")
else:
    print("❌ Não é possível executar Kraken2:")
    if not os.path.exists(kraken_db_expanded):
        print(f"   - Banco não encontrado: {kraken_db_expanded}")
    if not os.path.exists(nonhuman_r1):
        print(f"   - Arquivo R1 não encontrado: {nonhuman_r1}")
    if not os.path.exists(nonhuman_r2):
        print(f"   - Arquivo R2 não encontrado: {nonhuman_r2}")

Executando Kraken2 (isso pode demorar alguns minutos)...
Executando Kraken2...
  Database: /Users/larissa/kraken2_db/viral_db
  Input: patient_joao_nonhuman_R1.fastq.gz, patient_joao_nonhuman_R2.fastq.gz
Kraken2 concluído.
  Relatório: /Users/larissa/Desktop/TCC_metatrascriptomica/results/kraken2_reports/patient_joao_kraken2_report.txt
Kraken2 concluído!
⚠️ Bracken não está instalado ou não está no PATH.
   Usando resultados do Kraken2 apenas (isso é suficiente para o pipeline).
   Para instalar Bracken: conda install -c bioconda bracken

📊 Parseando resultados...
Total de táxons identificados: 211

Top 10 Espécies Detectadas:
                                                                         name  reads  percent
                        Severe acute respiratory syndrome-related coronavirus 334798    26.40
                                                         Pharaohvirus pharaoh  14324     1.13
                                                               Omarvirus omar   165